# Machine Learning Challenge

Below are 2 data challenges that test for your ability to:
- Wrangle/clean data to make it usable by a model
- Figure out how to set up X's and y's for a use case, given a dataset
- Write code to robustly and reproducibly preprocess data
- Pick/design the right model, and tune hyperparameters to get the best performance

You can use any programming language, model, and package to solve these problems. Let us know of any assumptions you make in your process.

#### Deliverables:
- A link to a github repository that contains:
    - Clearly commented code that was written to solve these problems
    - Your trained models stored in a file (`.pkl`, `.h5`, `.tar` - whatever is appropriate). The models must have `predict(X)` functions. 
    - A readme file that contains:
        - Instructions to easily access/load the above
        - A writeup explaining any significant design decisions and your reasons for making them. 
        - If needed, a brief writeup explaining anything you are particularly proud of in your implementation that you might want us to focus on

#### How we'll assess your work:
- Accuracy/RMSE of your model when predicting on held-out data
- How well various edge cases are handled when testing on held-out data. For example, if the held-out data contains:
    - A new column that wasn't present in the dataset given to you
    - New value in a categorical field that wasn't seen in the dataset given to you
    - NA values
- Efficiency of the code. 
    - Is it easy to understand? 
    - Are the variable names descriptive? 
    - Are there any variables created that aren't used? 
    - Is redundant code replaced with function calls? 
    - Is vectorized implementation used instead of nested for loops? 
    - Are classes defined and objects created where applicable? 
    - Are packages used to perform tasks instead of implementing them from scratch?
    
**NOTE:** Your stored models, once loaded, should *just work* when fed with our held-out data (which looks similar to the data we've given you). We won't do any preprocessing before we feed it into the model's `predict(X)` function; `predict(X)` should handle the preprocessing. Pay particular attention to handling the edge cases we've talked about.

Feel free to ask questions to clarify things. Submit everything you tried, not just the things that worked. I encourage you to try and showcase your talents. The more you go above and beyond what's expected, the more impressed we'll be. **Bonus points if you fit Keras/Tensorflow/Pytorch/Caffe models** in addition to your Linear/Tree-based models.

## 0. Import dependencies

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing as scale
from sklearn.utils import resample
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import mean_squared_error, accuracy_score, average_precision_score, precision_score, f1_score,recall_score, roc_auc_score

## Task 1
`predictive_maintenance_dataset.csv` is a file that contains parameters and settings (`operational_setting_1`, `operational_setting_2`, `sensor_measurement_1`, `sensor_measurement_2`, etc.) for many wind turbines. There is a column called `unit_number` which specifies which turbine it is, and one called `status`, in which a value of 1 means the turbine broke down that day, and 0 means it didn't. Your task is to create a model that, when fed with operational settings and sensor measurements (`unit_number` and `time_stamp` will *not* be fed in), outputs 1 if the turbine will break down within the next 40 days, and 0 if not.

**NOTE:** The model should output 1 if the turbine is anywhere between 40 and 0 days away from failure, not *only* 40 days from failure.

In [2]:
## What the data that we'll feed into your model's predict(X) function will look like:
# Notice what the operational_setting_3 column looks like
df_X = pd.read_csv("predictive_maintenance_dataset.csv").drop(labels=['status', 'unit_number', 'time_stamp'], axis='columns')
df_X

,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,sensor_measurement_7,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
0,42.0007,0.8415,High,445.00,NaN,1362.47,1143.17,3.91,5.70,142.53,...,133.75,2388.50,8129.92,9.1182,NaN,332.0,2212.0,100.00,10.77,6.5717
1,-0.0023,0.0004,High,518.67,642.33,1581.03,1400.06,14.62,21.61,554.60,...,522.19,2388.00,8135.70,8.3817,0.03,393.0,2388.0,100.00,39.07,23.3958
2,NaN,0.6216,Low,462.54,536.71,1250.87,1037.52,7.05,9.00,174.56,...,163.11,2028.06,7867.90,10.8827,NaN,306.0,1915.0,84.93,14.33,8.6202
3,42.0006,NaN,High,NaN,549.28,1349.42,1114.02,3.91,5.71,137.97,...,130.58,2387.71,8074.81,9.3776,0.02,NaN,2212.0,100.00,10.60,6.2614
4,-0.0016,0.0004,High,518.67,643.84,1604.53,1431.41,14.62,21.61,551.30,...,519.44,2388.24,8135.95,8.5223,0.03,396.0,2388.0,100.00,38.39,23.0682
5,25.0046,0.6219,Low,462.54,536.72,NaN,1047.79,7.05,9.03,175.36,...,164.97,2028.40,7880.19,10.8625,0.02,308.0,1915.0,84.93,14.38,8.6381
6,NaN,0.6200,Low,462.54,536.79,1267.31,1045.78,7.05,9.03,174.81,...,165.05,2028.37,7881.95,10.9150,0.02,307.0,1915.0,84.93,14.18,8.5752
7,42.0053,0.8400,High,445.00,548.84,1348.71,1119.73,3.91,5.71,138.95,...,130.38,2387.86,8079.78,9.3526,0.02,329.0,2212.0,100.00,10.64,6.5382
8,0.0029,-0.0003,High,NaN,642.48,1588.88,1393.88,14.62,21.61,NaN,...,522.01,2388.06,NaN,8.3743,0.03,392.0,2388.0,100.00,38.95,23.4351
9,10.0008,0.2504,High,489.05,604.49,1498.95,1309.51,10.52,15.49,394.85,...,371.56,2388.09,8128.11,NaN,0.03,368.0,2319.0,100.00,28.48,17.2737


### 1. Import data

In [62]:
df = pd.read_csv("predictive_maintenance_dataset.csv").sort_values(by = ['unit_number', 'time_stamp'], ascending = True).reset_index()
df

,index,unit_number,time_stamp,status,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
0,73382,2,2017-04-01 12:00:00,0,-0.0018,0.0006,High,518.67,641.89,1583.84,...,522.33,2388.06,8137.72,8.3905,0.03,391.0,2388.0,100.00,38.94,23.4585
1,90923,2,2017-04-02 12:00:00,0,0.0043,-0.0003,High,518.67,641.82,1587.05,...,522.70,2387.98,8131.09,8.4167,0.03,NaN,2388.0,100.00,39.06,23.4085
2,82527,2,2017-04-03 12:00:00,0,0.0018,0.0003,High,518.67,641.55,1588.32,...,522.58,2387.99,8140.58,8.3802,0.03,391.0,2388.0,100.00,39.11,23.4250
3,96521,2,2017-04-04 12:00:00,0,0.0035,-0.0004,High,518.67,641.68,1584.15,...,522.49,2387.93,8140.44,8.4018,0.03,391.0,2388.0,100.00,39.13,23.5027
4,73137,2,2017-04-05 12:00:00,0,0.0005,0.0004,High,518.67,641.73,1579.03,...,522.27,2387.94,8136.67,8.3867,0.03,390.0,2388.0,100.00,39.18,23.4234
5,6093,2,2017-04-06 12:00:00,0,-0.0010,0.0004,High,518.67,641.30,1577.50,...,522.80,2387.99,8133.65,8.3800,0.03,392.0,2388.0,100.00,39.15,23.4270
6,91573,2,2017-04-07 12:00:00,0,0.0001,-0.0002,High,518.67,642.03,1587.49,...,522.14,2388.04,8136.33,8.3941,0.03,391.0,2388.0,100.00,39.10,23.4718
7,77471,2,2017-04-08 12:00:00,0,0.0015,-0.0004,High,518.67,642.55,1590.41,...,522.77,NaN,NaN,8.3861,0.03,391.0,2388.0,100.00,NaN,23.4381
8,93541,2,2017-04-09 12:00:00,0,0.0017,-0.0004,High,518.67,641.98,1581.99,...,522.40,2387.98,8145.29,8.3868,0.03,390.0,2388.0,100.00,39.06,23.4875
9,30788,2,2017-04-10 12:00:00,0,NaN,0.0002,High,518.67,NaN,1586.37,...,521.99,2387.97,8138.64,8.3982,0.03,391.0,2388.0,100.00,NaN,23.6005


In [79]:
df.groupby('unit_number').count().where(df['unit_number']<40) # No unit number values less than 40

,index,time_stamp,status,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
unit_number,,,,,,,,,,,,,,,,,,,,,
2,287,287,287,275,271,278,275,271,279,274,...,280,279,280,273,269,275,269,276,273,270
3,179,179,179,176,170,176,173,167,175,169,...,166,164,175,168,165,171,168,172,172,172
4,189,189,189,178,180,177,179,179,176,179,...,184,184,178,174,187,175,183,183,181,184
5,269,269,269,254,253,249,256,254,257,258,...,252,255,260,258,256,254,256,260,258,258
6,188,188,188,183,175,178,177,178,174,176,...,179,172,177,180,181,180,178,179,179,175
7,259,259,259,247,252,246,246,246,243,249,...,237,247,250,248,250,253,245,250,243,242
8,150,150,150,142,138,146,142,146,140,138,...,147,138,139,144,144,139,142,146,143,139
9,201,201,201,190,193,190,188,188,191,185,...,189,193,191,186,194,189,189,191,194,191
10,222,222,222,209,211,213,213,212,211,217,...,213,217,213,217,209,211,211,215,215,213


### 2. Explore data

Are there any null columns? 

In [4]:
# Check for columns with Null values
nullcols = []

for col in df.columns:
    nbnull = (df[col].isnull()*1).sum()
    if (nbnull>0): 
        t = type(df[df[col].notnull()][col].iat[0]) # type of first non-null value
        nullcols.append([col,t])
        print(col, nbnull, t)

operational_setting_1 7141 <class 'numpy.float64'>
operational_setting_2 7196 <class 'numpy.float64'>
operational_setting_3 7227 <class 'str'>
sensor_measurement_1 7209 <class 'numpy.float64'>
sensor_measurement_2 7198 <class 'numpy.float64'>
sensor_measurement_3 7190 <class 'numpy.float64'>
sensor_measurement_4 7335 <class 'numpy.float64'>
sensor_measurement_5 7244 <class 'numpy.float64'>
sensor_measurement_6 7444 <class 'numpy.float64'>
sensor_measurement_7 7213 <class 'numpy.float64'>
sensor_measurement_8 7276 <class 'numpy.float64'>
sensor_measurement_9 7207 <class 'numpy.float64'>
sensor_measurement_10 7191 <class 'numpy.float64'>
sensor_measurement_11 7180 <class 'numpy.float64'>
sensor_measurement_12 7227 <class 'numpy.float64'>
sensor_measurement_13 7115 <class 'numpy.float64'>
sensor_measurement_14 7068 <class 'numpy.float64'>
sensor_measurement_15 7257 <class 'numpy.float64'>
sensor_measurement_16 7059 <class 'numpy.float64'>
sensor_measurement_17 7167 <class 'numpy.float64'>

That's a lot of empty values! 

### Categorical value

 Consider replacement with mode and creating dummy variables 

In [5]:
df['operational_setting_3'].fillna(df['operational_setting_3'].mode()[0], inplace=True)

Converting to dummy variable to numerically quantify categories
and further reduce variables by only including the high column to indicate if the load is high or low (1 or 0)

In [6]:
df = pd.concat([df.drop('operational_setting_3', axis=1), pd.get_dummies(df.operational_setting_3)], axis=1)

### Numerical values

Some 7000 values are missing out of 144000, that's about 5%, a significant number. This could be valuable information that otherwise may skew our data if not used. 

There are several ways we can approach the missing numerical values. We could use the mean or median values for the entire data set, or narrow down to those values of the individual units. 

Interestingly, the 'status' column gives us a score of how often each turbine breaks down. 

So now we just fill in the missing nan's with the average values of each individual tubine

In [7]:
df.fillna(df.mean(axis=0), axis=0, inplace=True)

,unit_number,time_stamp,status,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
73382,2,2017-04-01 12:00:00,0,-0.001800,0.000600,518.670000,641.890000,1583.840000,1391.2800,14.620000,...,8137.720000,8.39050,0.030000,391.000000,2388.000000,100.000000,38.940000,23.4585,1,0
90923,2,2017-04-02 12:00:00,0,0.004300,-0.000300,518.670000,641.820000,1587.050000,1393.1300,14.620000,...,8131.090000,8.41670,0.030000,360.999569,2388.000000,100.000000,39.060000,23.4085,1,0
82527,2,2017-04-03 12:00:00,0,0.001800,0.000300,518.670000,641.550000,1588.320000,1398.9600,14.620000,...,8140.580000,8.38020,0.030000,391.000000,2388.000000,100.000000,39.110000,23.4250,1,0
96521,2,2017-04-04 12:00:00,0,0.003500,-0.000400,518.670000,641.680000,1584.150000,1396.0800,14.620000,...,8140.440000,8.40180,0.030000,391.000000,2388.000000,100.000000,39.130000,23.5027,1,0
73137,2,2017-04-05 12:00:00,0,0.000500,0.000400,518.670000,641.730000,1579.030000,1402.5200,14.620000,...,8136.670000,8.38670,0.030000,390.000000,2388.000000,100.000000,39.180000,23.4234,1,0
6093,2,2017-04-06 12:00:00,0,-0.001000,0.000400,518.670000,641.300000,1577.500000,1396.7600,14.620000,...,8133.650000,8.38000,0.030000,392.000000,2388.000000,100.000000,39.150000,23.4270,1,0
91573,2,2017-04-07 12:00:00,0,0.000100,-0.000200,518.670000,642.030000,1587.490000,1400.6500,14.620000,...,8136.330000,8.39410,0.030000,391.000000,2388.000000,100.000000,39.100000,23.4718,1,0
77471,2,2017-04-08 12:00:00,0,0.001500,-0.000400,518.670000,642.550000,1590.410000,1262.2954,14.620000,...,8089.449809,8.38610,0.030000,391.000000,2388.000000,100.000000,26.050472,23.4381,1,0
93541,2,2017-04-09 12:00:00,0,0.001700,-0.000400,518.670000,641.980000,1581.990000,1395.0100,14.620000,...,8145.290000,8.38680,0.030000,390.000000,2388.000000,100.000000,39.060000,23.4875,1,0
30788,2,2017-04-10 12:00:00,0,17.075966,0.000200,518.670000,597.771761,1586.370000,1394.8600,14.620000,...,8138.640000,8.39820,0.030000,391.000000,2388.000000,100.000000,26.050472,23.6005,1,0


### Setting Labels

We have an interesting case here: where we're checking if a turbine is going to fail in 40 days or less. So essentially we're trying to figure out a problem where given all the parameters what is the likelihood that a certain unit fails within a 40 day timespan. 


So we just have to identify the date the turbines failed and mark any other data point going back up to a maximum of 40 days as a failure as well.

In [8]:
df.groupby(['status']).count()

,unit_number,time_stamp,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
status,,,,,,,,,,,,,,,,,,,,,
0,143570,143570,143570,143570,143570,143570,143570,143570,143570,143570,...,143570,143570,143570,143570,143570,143570,143570,143570,143570,143570
1,633,633,633,633,633,633,633,633,633,633,...,633,633,633,633,633,633,633,633,633,633


In [9]:
df['status'] = df['status'].replace(0, np.NaN) #Let's replace all the 0s with NaNs and then we work backwords

In [10]:
df['status'] = df['status'].fillna(method='bfill', limit=40) # fill backward up to 40days. Thankfully the data is frequent and daily
df['status'] = df['status'].fillna(int(0)) #fill the rest with zeros

In [11]:
df.groupby(['status']).count()

,unit_number,time_stamp,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
status,,,,,,,,,,,,,,,,,,,,,
0.0,118250,118250,118250,118250,118250,118250,118250,118250,118250,118250,...,118250,118250,118250,118250,118250,118250,118250,118250,118250,118250
1.0,25953,25953,25953,25953,25953,25953,25953,25953,25953,25953,...,25953,25953,25953,25953,25953,25953,25953,25953,25953,25953


In [12]:
df = df.drop(['time_stamp', 'unit_number'], axis = 1)


In [13]:
df_work = df #saving the filled dataframe in a separate variable to work

In [14]:
df 

,status,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,sensor_measurement_7,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
73382,0.0,-0.001800,0.000600,518.670000,641.890000,1583.840000,1391.2800,14.620000,21.600000,554.530000,...,8137.720000,8.39050,0.030000,391.000000,2388.000000,100.000000,38.940000,23.4585,1,0
90923,0.0,0.004300,-0.000300,518.670000,641.820000,1587.050000,1393.1300,14.620000,21.610000,554.770000,...,8131.090000,8.41670,0.030000,360.999569,2388.000000,100.000000,39.060000,23.4085,1,0
82527,0.0,0.001800,0.000300,518.670000,641.550000,1588.320000,1398.9600,14.620000,21.600000,555.140000,...,8140.580000,8.38020,0.030000,391.000000,2388.000000,100.000000,39.110000,23.4250,1,0
96521,0.0,0.003500,-0.000400,518.670000,641.680000,1584.150000,1396.0800,14.620000,21.610000,554.250000,...,8140.440000,8.40180,0.030000,391.000000,2388.000000,100.000000,39.130000,23.5027,1,0
73137,0.0,0.000500,0.000400,518.670000,641.730000,1579.030000,1402.5200,14.620000,14.487827,555.120000,...,8136.670000,8.38670,0.030000,390.000000,2388.000000,100.000000,39.180000,23.4234,1,0
6093,0.0,-0.001000,0.000400,518.670000,641.300000,1577.500000,1396.7600,14.620000,21.610000,554.980000,...,8133.650000,8.38000,0.030000,392.000000,2388.000000,100.000000,39.150000,23.4270,1,0
91573,0.0,0.000100,-0.000200,518.670000,642.030000,1587.490000,1400.6500,14.620000,21.610000,554.840000,...,8136.330000,8.39410,0.030000,391.000000,2388.000000,100.000000,39.100000,23.4718,1,0
77471,0.0,0.001500,-0.000400,518.670000,642.550000,1590.410000,1262.2954,14.620000,21.610000,554.650000,...,8089.449809,8.38610,0.030000,391.000000,2388.000000,100.000000,26.050472,23.4381,1,0
93541,0.0,0.001700,-0.000400,518.670000,641.980000,1581.990000,1395.0100,14.620000,21.600000,554.690000,...,8145.290000,8.38680,0.030000,390.000000,2388.000000,100.000000,39.060000,23.4875,1,0
30788,0.0,17.075966,0.000200,518.670000,597.771761,1586.370000,1394.8600,14.620000,21.600000,554.630000,...,8138.640000,8.39820,0.030000,391.000000,2388.000000,100.000000,26.050472,23.6005,1,0


In [15]:
columns_to_normalize = df.columns[1:24] 



standard_sc = scale.StandardScaler()
x_std = standard_sc.fit_transform(df)
df_scaled = pd.DataFrame(x_std)

In [16]:
df_scaled.describe()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
count,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,...,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05
mean,-4.815160e-14,-2.407497e-15,2.686961e-14,1.197724e-13,5.664696e-15,1.888140e-15,-4.368540e-15,-1.840133e-15,8.059762e-14,1.098348e-15,...,-6.605802e-15,-1.318948e-15,-1.287545e-14,-1.627356e-14,2.725349e-15,6.726376e-14,-1.380750e-15,6.966404e-16,-4.265057e-14,4.265057e-14
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,...,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-4.684823e-01,-1.059865e+00,-1.133609e+00,-1.386581e+00,-1.503193e+00,-1.954478e+00,-1.795655e+00,-1.448505e+00,-1.404439e+00,-1.324699e+00,...,-3.070024e+00,-1.220100e+00,-1.072788e+00,-1.951242e+00,-2.590864e+00,-2.978932e+00,-1.392082e+00,-1.393568e+00,-2.985677e+00,-3.349324e-01
25%,-4.684823e-01,-1.059239e+00,-1.131381e+00,-1.236909e+00,-1.148943e+00,-9.492222e-01,-1.010243e+00,-1.071113e+00,-1.033334e+00,-1.090434e+00,...,-2.183847e-01,-8.299699e-01,-1.072788e+00,-9.590779e-01,-4.510524e-01,3.533196e-01,-1.021514e+00,-1.022292e+00,3.349324e-01,-3.349324e-01
50%,-4.684823e-01,1.388497e-14,-4.858593e-13,9.834253e-02,1.790132e-01,1.805076e-01,3.252236e-14,5.926707e-13,8.985701e-13,8.364575e-15,...,2.641044e-01,-1.455905e-14,3.527769e-12,1.653750e-01,3.198561e-01,3.533196e-01,-1.774039e-14,-4.930500e-15,3.349324e-01,-3.349324e-01
75%,-4.684823e-01,1.111943e+00,1.207995e+00,1.096831e+00,1.074774e+00,1.024485e+00,1.047802e+00,1.125938e+00,1.134367e+00,1.130136e+00,...,6.300779e-01,3.935588e-01,9.801296e-01,1.025251e+00,8.169840e-01,3.533196e-01,1.117796e+00,1.118489e+00,3.349324e-01,-3.349324e-01
max,2.134552e+00,1.546685e+00,1.213567e+00,1.096831e+00,1.142343e+00,1.291315e+00,1.346519e+00,1.125938e+00,1.134367e+00,1.233613e+00,...,2.601165e+00,2.755787e+00,9.801296e-01,1.256756e+00,8.169840e-01,3.533196e-01,1.212410e+00,1.215329e+00,3.349324e-01,2.985677e+00


In [17]:
df['status']

73382     0.0
90923     0.0
82527     0.0
96521     0.0
73137     0.0
6093      0.0
91573     0.0
77471     0.0
93541     0.0
30788     0.0
34774     0.0
130580    0.0
49503     0.0
35236     0.0
121382    0.0
101000    0.0
7046      0.0
25719     0.0
118022    0.0
54533     0.0
125449    0.0
40954     0.0
44034     0.0
38522     0.0
64346     0.0
41008     0.0
82368     0.0
116847    0.0
10153     0.0
27089     0.0
         ... 
80076     1.0
74145     1.0
3518      1.0
1029      1.0
88108     1.0
73894     1.0
112471    1.0
91021     1.0
6049      1.0
138888    1.0
112944    1.0
143832    1.0
2075      1.0
4067      1.0
38296     1.0
118147    1.0
84389     1.0
25322     1.0
102080    1.0
24077     1.0
136362    1.0
91459     1.0
98734     1.0
51634     1.0
86092     1.0
121541    1.0
101889    1.0
17932     1.0
10097     1.0
103638    1.0
Name: status, Length: 144203, dtype: float64

### Modelling

In [36]:
xtrain, xtest, ytrain, ytest = train_test_split(df_scaled, df['status'], test_size = 0.40, random_state = 19 )

In [37]:
ytrain=ytrain.astype(int)
ytest=ytest.astype(int)

In [38]:
xtrain

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
56935,-0.468482,1.546139e+00,1.212731e+00,-1.386581e+00,-1.161009e+00,-9.747421e-01,-1.042388e+00,-1.448505e+00,-1.396475e+00,-1.307511e+00,...,6.227142e-02,4.079019e-01,-1.072788e+00,-0.992150,-4.510524e-01,0.353320,-1.343023e+00,-1.353482e+00,0.334932,-0.334932
34648,2.134552,1.388497e-14,8.205136e-01,1.704818e-01,2.557514e-01,1.936147e-01,-2.947572e-02,-1.408518e-01,-1.318502e-01,-1.593202e-01,...,-5.478118e-01,2.637886e-01,-1.072788e+00,0.198447,-3.276342e-15,0.353320,-1.664905e-01,-1.705694e-01,0.334932,-0.334932
48974,-0.468482,-4.387355e-01,-4.355290e-01,9.834253e-02,-1.124808e-13,2.527269e-01,3.293159e-01,1.403899e-01,1.596188e-01,1.956083e-01,...,6.495609e-01,-5.709758e-01,9.801296e-01,0.297664,3.198561e-01,0.353320,2.391199e-01,2.220208e-01,0.334932,-0.334932
115802,-0.468482,1.546617e+00,1.207995e+00,-1.386581e+00,-1.165594e+00,-9.619822e-01,-1.031698e+00,-1.448505e+00,-1.398068e+00,-1.311867e+00,...,-1.477113e-01,4.394565e-01,3.527769e-12,-0.959078,-4.510524e-01,0.353320,-1.363173e+00,-1.350166e+00,0.334932,-0.334932
21091,-0.468482,-1.059226e+00,-1.129152e+00,1.096831e+00,1.067776e+00,1.012593e+00,1.069785e+00,1.125938e+00,1.134367e+00,1.138141e+00,...,5.940409e-01,-9.067392e-01,9.801296e-01,0.992179,8.169840e-01,0.353320,1.141450e+00,1.136589e+00,0.334932,-0.334932
26272,-0.468482,-4.388658e-01,-4.355290e-01,9.834253e-02,-1.124808e-13,4.539400e-14,3.281867e-01,1.403899e-01,1.596188e-01,1.959026e-01,...,7.485036e-01,-5.732980e-01,9.801296e-01,0.297664,3.198561e-01,0.353320,2.233510e-01,2.061267e-01,0.334932,-0.334932
121512,-0.468482,-4.386983e-01,-4.355290e-01,9.834253e-02,1.531925e-01,2.488209e-01,2.836953e-01,1.403899e-01,1.548406e-01,1.931362e-01,...,4.114361e-01,-6.155074e-01,9.801296e-01,0.297664,3.198561e-01,0.353320,-1.774039e-14,2.309028e-01,0.334932,-0.334932
107347,-0.468482,1.817926e-01,8.180065e-01,1.704818e-01,2.415138e-01,2.103675e-01,-7.449405e-02,-1.408518e-01,-1.318502e-01,-1.581430e-01,...,-3.425406e-01,2.756728e-01,-1.072788e+00,0.132303,3.558798e-01,0.353320,-1.774039e-14,-1.369843e-01,0.334932,-0.334932
106275,-0.468482,1.388497e-14,1.213288e+00,-1.386581e+00,-1.151115e+00,-9.151090e-01,-9.515988e-01,-1.448505e+00,-1.396475e+00,-1.308747e+00,...,6.494554e-02,5.017463e-01,-1.072788e+00,-0.992150,-4.510524e-01,0.353320,-1.353536e+00,-4.930500e-15,0.334932,-0.334932
44548,2.134552,-1.059251e+00,-1.131938e+00,1.096831e+00,-1.124808e-13,1.080646e+00,3.252236e-14,1.125938e+00,1.134367e+00,1.126133e+00,...,8.917719e-13,-7.844820e-01,9.801296e-01,1.124467,8.169840e-01,0.353320,1.109912e+00,1.095627e+00,0.334932,-0.334932


In [39]:
xtest

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
112254,2.134552,1.546083e+00,1.207995e+00,-1.386581e+00,-1.141945e+00,-9.639786e-01,-9.174210e-01,-1.448505e+00,-1.398068e+00,-1.314339,...,-9.868562e-02,5.201873e-01,-1.072788e+00,-0.992150,-4.510524e-01,3.533196e-01,-1.350908e+00,-1.366995e+00,0.334932,-0.334932
101539,-0.468482,1.546319e+00,1.213567e+00,-1.386581e+00,-1.172592e+00,-1.065798e+00,-1.090794e+00,-1.448505e+00,-1.399661e+00,-1.311631,...,-1.855311e-01,4.688256e-01,-1.072788e+00,-1.025222,-4.510524e-01,3.533196e-01,-1.360544e+00,-1.340758e+00,0.334932,-0.334932
72074,-0.468482,1.388497e-14,-1.131381e+00,1.096831e+00,1.072120e+00,1.023530e+00,1.074979e+00,5.926707e-13,1.129589e+00,1.114950,...,8.917719e-13,-8.260085e-01,9.801296e-01,0.992179,8.169840e-01,3.533196e-01,-1.774039e-14,1.119147e+00,0.334932,-0.334932
79251,-0.468482,-1.059394e+00,-1.131659e+00,1.096831e+00,1.084910e+00,9.907187e-01,1.010689e+00,1.125938e+00,8.985701e-13,1.132373,...,5.318992e-01,-9.395233e-01,9.801296e-01,1.058323,8.169840e-01,3.533196e-01,1.140574e+00,1.129227e+00,0.334932,-0.334932
102502,-0.468482,1.111961e+00,1.209945e+00,-1.236909e+00,-1.027562e+00,-1.023525e+00,3.252236e-14,-1.071113e+00,-1.036519e+00,-0.986604,...,-3.762856e-01,3.380991e-01,-1.072788e+00,-0.959078,-3.276342e-15,3.533196e-01,-9.838437e-01,-9.916723e-01,0.334932,-0.334932
55949,2.134552,-1.059326e+00,-1.131938e+00,1.096831e+00,1.091425e+00,1.119273e+00,1.212744e+00,1.125938e+00,1.134367e+00,1.124191,...,1.005347e+00,-7.837990e-01,9.801296e-01,1.157539,8.169840e-01,3.533196e-01,1.081878e+00,1.092822e+00,0.334932,-0.334932
65426,2.134552,1.388497e-14,-1.131938e+00,2.318589e-13,1.089012e+00,1.214061e+00,1.164941e+00,1.125938e+00,8.985701e-13,1.127781,...,1.402264e+00,-7.682266e-01,9.801296e-01,1.124467,8.169840e-01,-4.135239e-12,1.101152e+00,1.089520e+00,0.334932,-0.334932
69266,-0.468482,-1.059146e+00,-1.131102e+00,1.096831e+00,1.088529e+00,1.053911e+00,1.130010e+00,1.125938e+00,1.134367e+00,1.130077,...,8.282181e-01,-8.096164e-01,9.801296e-01,1.058323,8.169840e-01,3.533196e-01,1.109036e+00,1.096240e+00,0.334932,-0.334932
115518,-0.468482,1.388497e-14,1.207995e+00,-1.236909e+00,-1.019598e+00,-8.304769e-01,-9.549864e-01,-1.071113e+00,-1.033334e+00,-0.980718,...,-2.917322e-01,3.870020e-01,-1.072788e+00,-0.992150,-3.718001e-01,3.533196e-01,-9.890999e-01,-9.755737e-01,0.334932,-0.334932
114311,-0.468482,-1.059276e+00,-1.131938e+00,1.096831e+00,1.079842e+00,9.994857e-01,1.013700e+00,1.125938e+00,1.132775e+00,1.151149,...,6.857253e-01,-9.258633e-01,9.801296e-01,0.992179,8.169840e-01,3.533196e-01,1.144078e+00,1.132762e+00,0.334932,-0.334932


In [50]:
def logisticRegression(xtrain,xtest,ytrain,ytest):
    LR = LogisticRegression(multi_class='ovr')
    model = LR.fit(xtrain.values,ytrain.values)
    pred = model.predict(xtest.values)
    metrics(pred,ytest)


In [51]:
logisticRegression(xtrain,xtest,ytrain,ytest)

accuracy score:  1.0
RMSE: 0.0
Recall score:  1.0
average_precision_score:  1.0
Precision Score:  1.0
F1_score:  1.0
roc_auc_score:  1.0


In [45]:
randomForestClassifier(xtrain,xtest,ytrain,ytest,n_estimators=25,min_samples_split=25,max_depth=5,random_state=72)

accuracy score:  1.0
RMSE: 0.0
Recall score:  1.0
average_precision_score:  1.0
Precision Score:  1.0
F1_score:  1.0
roc_auc_score:  1.0


In [46]:
supportVectorMachine(xtrain,xtest,ytrain,ytest)

accuracy score:  1.0
RMSE: 0.0
Recall score:  1.0
average_precision_score:  1.0
Precision Score:  1.0
F1_score:  1.0
roc_auc_score:  1.0


In [49]:
gaussianNaiveBayes(xtrain,xtest,ytrain,ytest)

In [21]:
def decisionTreeClassifier(xtrain, xtest, ytrain, ytest, random_state):
    DT = decisionTreeClassifier(xtrain, xtest, ytrain, ytest, random_state=12)
    model = DT.fit(xtrain,ytrain)
    pred = model.predict(xtest)
    metrics(ytest,pred)
    

In [22]:
def randomForestClassifier(xtrain,xtest,ytrain,ytest,n_estimators=25,min_samples_split=25,max_depth=5,random_state=72):
    RF = RandomForestClassifier(n_estimators = 25, min_samples_split=25, max_depth =5, random_state=72)
    model = RF.fit(xtrain,ytrain)
    pred = RF.predict(xtest)
    metrics(ytest, pred)

In [23]:
def gaussianNaiveBayes(xtrain,xtest,ytrain,ytest):
    GNB = GaussianNB()
    model = GNB.fit(xtrain,ytrain)
    pred = GNB.predict(xtest)
    metrics = (ytest, pred)

In [24]:
def supportVectorMachine(xtrain,xtest,ytrain,ytest):
    svc = SVC(kernel='linear')
    model = svc.fit(xtrain,ytrain)
    pred = svc.predict(xtest)
    metrics(ytest,pred)

In [25]:
def metrics(ytest, pred):
    """
    Function to evaluate models against models 
    """
    print('accuracy score: ', accuracy_score(ytest, pred))
    print('RMSE:', mean_squared_error(ytest,pred))
    print('Recall score: ', recall_score(ytest,pred))
    
    print('average_precision_score: ', average_precision_score(ytest,pred))
    print('Precision Score: ',precision_score(ytest,pred))
    print('F1_score: ',f1_score(ytest, pred))
    print('roc_auc_score: ', roc_auc_score(ytest, pred))

### Conclusion

100% on everything? That's very fishy! Perhaps, my way of marking due for repair might be doing something to the dataset?
or the extent of data scaling might be? 

## Task 2
`forecasting_dataset.csv` is a file that contains pollution data for a city. Your task is to create a model that, when fed with columns `co_gt`, `nhmc`, `c6h6`, `s2`, `nox`, `s3`, `no2`, `s4`, `s5`, `t`, `rh`, `ah`, and `level`, predicts the value of `y` six hours later.

**NOTE:** In the data we've given you, the value of `y` for a given row is the value of `y` *for the timestamp of that same row*. We're asking you to predict the value of `y` 6 hours *after the timestamp of that row*.

In [100]:
## What the data that we'll feed into your model's predict(X) function will look like:
# Notice what the level column looks like
pd.read_csv("forecasting_dataset.csv").head().drop(labels=['date', 'time', 'y'], axis='columns')

,co_gt,nhmc,c6h6,s2,nox,s3,no2,s4,s5,t,rh,ah,level
0,-200.0,-200.0,7.2,867.0,-200.0,834.0,-200.0,1314.0,891.0,14.8,57.3,0.9603,NaN
1,0.5,-200.0,3.9,704.0,-200.0,861.0,-200.0,1603.0,860.0,24.4,65.0,1.9612,Low
2,3.7,-200.0,23.3,1386.0,NaN,626.0,109.0,2138.0,NaN,23.3,38.6,1.0919,High
3,2.1,-200.0,12.1,1052.0,183.0,779.0,NaN,1690.0,952.0,28.5,27.3,1.0479,High
4,4.4,-200.0,21.7,1342.0,786.0,499.0,206.0,1546.0,2006.0,12.9,54.1,0.8003,High


In [82]:
pd.read_csv("forecasting_dataset.csv").sort_values(by=['date'], ascending = False)

,date,time,y,co_gt,nhmc,c6h6,s2,nox,s3,no2,s4,s5,t,rh,ah,level
1820,9/9/2004,3:00:00,814,NaN,-200.0,2.3,606.0,-200.0,1370.0,-200.0,1131.0,NaN,23.0,33.0,0.9184,Low
3802,9/9/2004,19:00:00,1329,4.3,-200.0,22.3,1360.0,435.0,NaN,175.0,1768.0,1894.0,31.2,25.5,1.1379,High
1545,9/9/2004,10:00:00,1277,3.1,-200.0,19.9,1292.0,590.0,800.0,171.0,1626.0,1481.0,31.0,21.9,0.9708,High
5954,9/9/2004,8:00:00,1289,3.2,NaN,21.4,1335.0,542.0,724.0,130.0,1681.0,1521.0,21.0,35.6,NaN,High
2172,9/9/2004,18:00:00,1425,5.4,-200.0,32.3,1605.0,631.0,660.0,220.0,1920.0,2321.0,35.2,16.6,0.9242,High
7545,9/9/2004,14:00:00,1160,2.5,-200.0,16.2,NaN,NaN,873.0,168.0,1522.0,1162.0,35.1,17.5,0.9733,High
5836,9/9/2004,22:00:00,1089,1.5,-200.0,11.3,1024.0,180.0,831.0,111.0,1452.0,NaN,27.4,31.5,1.1330,High
2564,9/9/2004,9:00:00,1355,3.5,-200.0,21.3,1333.0,590.0,741.0,157.0,1691.0,1605.0,25.0,28.9,0.9025,High
1764,9/9/2004,16:00:00,1166,2.9,-200.0,16.2,1186.0,393.0,878.0,175.0,1500.0,1180.0,38.4,13.8,0.9221,High
6216,9/9/2004,15:00:00,1148,2.4,-200.0,15.0,1149.0,NaN,924.0,152.0,1473.0,1047.0,35.4,16.6,NaN,NaN
